# Preliminaries
We need to load a few libraries first

In [1]:
import pandas as pd
import numpy as np
import ampligraph
import stardog
import json
import io
import tensorflow as tf
print(tf.__version__)
print(ampligraph.__version__)

2.12.0
2.0.0


# Loading the data

In [2]:
df = pd.read_csv("graph_content.csv.gz").dropna()
df.head()

,s,p,o
0,tag:stardog:designer:Tutorial:data:Match:3249,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:8472
1,tag:stardog:designer:Tutorial:data:Match:3249,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:9825
2,tag:stardog:designer:Tutorial:data:Match:3250,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:10260
3,tag:stardog:designer:Tutorial:data:Match:3250,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:8668
4,tag:stardog:designer:Tutorial:data:Match:3251,tag:stardog:designer:Tutorial:model:match_team,tag:stardog:designer:Tutorial:data:Team:9850


# Train the model

In [3]:
from ampligraph.evaluation import train_test_split_no_unseen 
test_size = int(df.shape[0]*0.10)
X_train, X_valid = train_test_split_no_unseen(np.array(df), test_size=test_size)
print('Train set size: ', X_train.shape)
print('Test set size: ', X_valid.shape)

Train set size:  (9090, 3)
Test set size:  (1010, 3)


In [4]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.latent_features.regularizers import get as get_regularizer
from ampligraph.latent_features.loss_functions import get as get_loss
import tensorflow as tf

# create the model
model = ScoringBasedEmbeddingModel(k=300, eta=20,
                                   scoring_type='ComplEx', seed=0)

# create reg, loss, optim
reg = get_regularizer('LP', hyperparams={'p':3, 'lambda':1e-3})
loss = get_loss('pairwise', hyperparams={'margin':5})
optim = tf.keras.optimizers.Adam(learning_rate = 1e-4)

# compile the model
model.compile(loss=loss, optimizer=optim, entity_relation_regularizer=reg)
model.fit(X_train, batch_size=5000, epochs=500)

WARNING - At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.
Epoch 1/500


2023-05-06 22:11:18.521762: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 2s 571ms/step - loss: 454498.2500
Epoch 2/500
3/3 [==============================] - 0s 142ms/step - loss: 454493.9062
Epoch 3/500
3/3 [==============================] - 0s 147ms/step - loss: 454490.1250
Epoch 4/500
3/3 [==============================] - 0s 145ms/step - loss: 454486.2500
Epoch 5/500
3/3 [==============================] - 0s 143ms/step - loss: 454482.3438
Epoch 6/500
3/3 [==============================] - 0s 143ms/step - loss: 454478.4688
Epoch 7/500
3/3 [==============================] - 0s 143ms/step - loss: 454474.5312
Epoch 8/500
3/3 [==============================] - 0s 143ms/step - loss: 454470.5000
Epoch 9/500
3/3 [==============================] - 0s 145ms/step - loss: 454466.3750
Epoch 10/500
3/3 [==============================] - 0s 147ms/step - loss: 454462.3125
Epoch 11/500
3/3 [==============================] - 0s 144ms/step - loss: 454458.0312
Epoch 12/500
3/3 [==============================] - 0s 145ms/step - loss: 4

In [7]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
ranks = model.evaluate(X_valid,
                       batch_size=5000,
                       use_filter={'train': X_train, 'test': X_valid},
                       verbose=True)
print('MR:', mr_score(ranks))
print('MRR:', mrr_score(ranks))
print('Hits@1:', hits_at_n_score(ranks, 1))
print('Hits@3:', hits_at_n_score(ranks, 3))
print('Hits@10:', hits_at_n_score(ranks, 10))
print('Hits@100:', hits_at_n_score(ranks, 100))

2/2 [==============================] - 2s 999ms/step
MR: 424.0564356435644
MRR: 0.2278293735294437
Hits@1: 0.18762376237623762
Hits@3: 0.2188118811881188
Hits@10: 0.31386138613861386
Hits@100: 0.5232673267326733


# Make predictions

We will now generate all the pairs of <player, plays for, team> and rank them by prediction score

In [8]:
players = set(df[df['s'].str.contains('Player:')]['s'].values) | \
          set(df[df['o'].str.contains('Player:')]['o'].values)
teams = set(df[df['s'].str.contains('Team:')]['s'].values) | \
        set(df[df['o'].str.contains('Team:')]['o'].values)
print('Number of players: ', len(players))
print('Number of teams: ', len(teams))

Number of players:  2344
Number of teams:  98


In [9]:
pairs = []
for player in players:
  for team in teams:
    pairs.append([player, 'tag:stardog:designer:Tutorial:model:plays_for', team])
# Create the possible combinations to test
df_to_test = pd.DataFrame(pairs, columns=['s', 'p', 'o'])
# Remove all the pairs we used for training
df_to_test = pd.merge(df_to_test, df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
print (df_to_test.shape)
df_to_test.head()

(227262, 3)


,s,p,o
0,tag:stardog:designer:Tutorial:data:Player:39968,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8529
1,tag:stardog:designer:Tutorial:data:Player:39968,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9783
2,tag:stardog:designer:Tutorial:data:Player:39968,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8543
3,tag:stardog:designer:Tutorial:data:Player:39968,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8533
4,tag:stardog:designer:Tutorial:data:Player:39968,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:10172


In [10]:
df_to_test['scores'] = model.predict(np.array(df_to_test))
df_to_test = df_to_test.sort_values(by='scores', ascending=False)
df_to_test

,s,p,o,scores
72979,tag:stardog:designer:Tutorial:data:Player:213489,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8634,2.684682
72957,tag:stardog:designer:Tutorial:data:Player:213489,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8315,2.646686
142452,tag:stardog:designer:Tutorial:data:Player:277439,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9837,2.623410
130790,tag:stardog:designer:Tutorial:data:Player:32577,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9837,2.615693
222803,tag:stardog:designer:Tutorial:data:Player:40543,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9853,2.595096
...,...,...,...,...
170833,tag:stardog:designer:Tutorial:data:Player:34305,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9878,-0.809048
144412,tag:stardog:designer:Tutorial:data:Player:141161,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9837,-0.809367
66561,tag:stardog:designer:Tutorial:data:Player:109058,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9878,-0.814619
55666,tag:stardog:designer:Tutorial:data:Player:46891,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8543,-0.837860


In [11]:
df_to_test[df_to_test.s == 'tag:stardog:designer:Tutorial:data:Player:30981']

,s,p,o,scores
89769,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9783,1.931672
89812,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8305,1.701925
89802,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8558,1.671044
89828,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8633,1.637627
89788,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:9906,1.583733
...,...,...,...,...
89779,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:10233,-0.231687
89851,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8466,-0.276657
89819,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:8592,-0.326318
89831,tag:stardog:designer:Tutorial:data:Player:30981,tag:stardog:designer:Tutorial:model:plays_for,tag:stardog:designer:Tutorial:data:Team:4087,-0.347767


# Improve the presentation
Figuring things out based on URIs is a bit challenging. We will return to Stardog to ask for the label associated to each of those identifiers and update the data frame with those.

For running the following cells you will need to create a file `connection_details.json` with this content in it:

```json
{
  "database": "YOUR DATABASE (e.g. Tutorial)",
  "endpoint": "YOUR ENDPOINT (e.g. https://sd-0fcedc52.stardog.cloud:5820)",
  "username": "YOUR USERNAME (e.g. christophe.gueret@accenture.com)",
  "password": "YOUR PASSWORD (e.g. Passw0rd)",
}
```

In [14]:
connection_details = json.load(open('connection_details.json'))
conn = stardog.Connection(**connection_details)

In [15]:
query = """
SELECT ?identifier ?label FROM stardog:context:all WHERE {
  ?identifier rdfs:label ?label .
}
"""
csv_results = conn.select(query, content_type='text/csv')
df_labels = pd.read_csv(io.BytesIO(csv_results)).set_index('identifier')
labels = df_labels.to_dict()['label']
print ('Loaded {} labels'.format(len(labels)))

Loaded 2673 labels


In [16]:
df_to_test.replace(labels)

,s,p,o,scores
72979,Alvaro Vazquez,plays for,FC Barcelona,2.684682
72957,Alvaro Vazquez,plays for,Athletic Club de Bilbao,2.646686
142452,Florian Raspentino,plays for,Stade de Reims,2.623410
130790,Bernard Mendy,plays for,Stade de Reims,2.615693
222803,Matthieu Chalme,plays for,AS Saint-Étienne,2.595096
...,...,...,...,...
170833,Maicon,plays for,Pescara,-0.809048
144412,Julian Schieber,plays for,Stade de Reims,-0.809367
66561,Jack Rodwell,plays for,Pescara,-0.814619
55666,Quentin Othon,plays for,Lazio,-0.837860


In [17]:
df_to_test[df_to_test.s == 'tag:stardog:designer:Tutorial:data:Player:30981'].replace(labels)

,s,p,o,scores
89769,Lionel Messi,plays for,RC Deportivo de La Coruña,1.931672
89812,Lionel Messi,plays for,Getafe CF,1.701925
89802,Lionel Messi,plays for,RCD Espanyol,1.671044
89828,Lionel Messi,plays for,Real Madrid CF,1.637627
89788,Lionel Messi,plays for,Atlético Madrid,1.583733
...,...,...,...,...
89779,Lionel Messi,plays for,Genoa,-0.231687
89851,Lionel Messi,plays for,Southampton,-0.276657
89819,Lionel Messi,plays for,Olympique de Marseille,-0.326318
89831,Lionel Messi,plays for,Évian Thonon Gaillard FC,-0.347767
